<a href="https://colab.research.google.com/github/KenyanGeek/Fraud-Detections-using-Fast.ai-Unbalanced-Data/blob/master/Xente_Fraud_Detection_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Fraud at Xende

In [0]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [37]:
from google.colab import files
uploaded = files.upload()


Saving newdata.xlsx to newdata (2).xlsx


In [39]:
import io


df_train = pd.read_excel(io.BytesIO(uploaded['newdata.xlsx']))

df_train


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0
5,TransactionId_23223,BatchId_25954,AccountId_1078,SubscriptionId_4238,CustomerId_1432,UGX,256,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2018-11-15T03:35:10Z,2,0
6,TransactionId_118063,BatchId_118460,AccountId_2442,SubscriptionId_1980,CustomerId_2858,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,10000.0,10000,2018-11-15T03:44:31Z,4,0
7,TransactionId_100640,BatchId_38561,AccountId_4841,SubscriptionId_3829,CustomerId_2858,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-500.0,500,2018-11-15T03:45:13Z,2,0
8,TransactionId_51905,BatchId_93774,AccountId_272,SubscriptionId_4731,CustomerId_598,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,500.0,500,2018-11-15T04:14:59Z,2,0
9,TransactionId_130161,BatchId_82409,AccountId_710,SubscriptionId_920,CustomerId_1053,UGX,256,ProviderId_1,ProductId_15,financial_services,ChannelId_3,600.0,600,2018-11-15T04:31:48Z,2,0


In [0]:
dep_var = 'FraudResult'


cat_names = ['TransactionId','BatchId','AccountId','SubscriptionId','CustomerId','CurrencyCode','CountryCode','ProviderId','ProductId','ProductCategory','ChannelId','TransactionStartTime','PricingStrategy']
cont_names = ['Amount','Value']
procs = [FillMissing, Categorify, Normalize]

In [42]:
from google.colab import files
uploaded = files.upload()


Saving test.csv to test (3).csv


In [0]:
import io


df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

test = TabularList.from_df(df_test, cat_names=cat_names, cont_names=cont_names, procs=procs) 


In [0]:
data = (TabularList.from_df(df_train, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_rand_pct(0.2)
                           .label_from_df(cols=dep_var)
                           .add_test(test, label = 'FraudResult')
                           .databunch())


In [45]:
data.show_batch(rows=10)

TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,TransactionStartTime,PricingStrategy,Amount,Value,target
TransactionId_139275,BatchId_30883,AccountId_3636,SubscriptionId_3125,CustomerId_4075,UGX,256,ProviderId_1,ProductId_15,financial_services,ChannelId_3,2019-01-01T06:39:50Z,2,-0.0353,-0.0425,1
TransactionId_87206,BatchId_66979,AccountId_4841,SubscriptionId_3829,CustomerId_625,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,2018-11-16T07:04:10Z,2,-0.3778,-0.3858,0
TransactionId_28205,BatchId_87787,AccountId_4841,SubscriptionId_3829,CustomerId_3427,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,2018-11-16T17:56:30Z,2,-0.3779,-0.3856,0
TransactionId_115979,BatchId_55743,AccountId_4421,SubscriptionId_4038,CustomerId_4878,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2018-12-17T16:51:31Z,2,0.0332,0.0262,1
TransactionId_87682,BatchId_101919,AccountId_4249,SubscriptionId_4429,CustomerId_7401,UGX,256,ProviderId_4,ProductId_3,airtime,ChannelId_2,2019-01-10T21:27:12Z,4,-0.7202,-0.0425,1
TransactionId_109763,BatchId_13161,AccountId_192,SubscriptionId_433,CustomerId_515,UGX,256,ProviderId_1,ProductId_15,financial_services,ChannelId_3,2018-11-16T06:31:54Z,2,-0.3572,-0.3652,0
TransactionId_49913,BatchId_133354,AccountId_1240,SubscriptionId_4672,CustomerId_1602,UGX,256,ProviderId_3,ProductId_15,financial_services,ChannelId_3,2019-01-28T21:10:27Z,2,0.9921,0.9876,1
TransactionId_98066,BatchId_90047,AccountId_1394,SubscriptionId_374,CustomerId_1758,UGX,256,ProviderId_6,ProductId_14,financial_services,ChannelId_3,2018-11-15T21:58:43Z,2,-0.3755,-0.3836,0
TransactionId_134550,BatchId_32834,AccountId_1208,SubscriptionId_1355,CustomerId_1567,UGX,256,ProviderId_3,ProductId_15,financial_services,ChannelId_3,2019-01-20T12:53:46Z,2,-0.0353,-0.0425,1
TransactionId_31560,BatchId_40311,AccountId_3081,SubscriptionId_4312,CustomerId_3507,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,2018-11-15T08:51:13Z,4,-0.3774,-0.3855,0


In [0]:
learn = tabular_learner(data, layers=[300,150], metrics=accuracy)

In [47]:
learn.fit(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.084219,0.323591,1.000000,00:01


## Inference

## Here i can predict with 99.77% accuracy

In [0]:
learn.predict(row)

(Category Good, tensor(1), tensor([0.2381, 0.7619]))

In [0]:

probs = learn.get_preds(ds_type=DatasetType.Test)[0]

def probs2class(item): 
    return max(range(len(item)), key=item.__getitem__) 

test_df = pd.DataFrame({'TransactionId': df_test['TransactionId'], 'target': list(map(probs2class, probs))})
test_df.to_csv('xende predictions.csv', header=True, index=False)

